In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import eznf
from eznf import Tensor
from eznf import datasets
from eznf.autograd import function
# from eznf.nn.modules import *

In [ ]:
class test(eznf.nn.Module):
    def __init__(self):
        super().__init__()
        self.networks = [
            eznf.nn.Linear(784, 256),
            eznf.nn.ReLU(),
            eznf.nn.Linear(256, 10)
        ]
    
    def forward(self, x):
        for i in self.networks:
            x = i(x)
        return x

In [ ]:
dataset = datasets.MNIST('./', False)
data = dataset.get()

In [ ]:
X_train, Y_train, X_test, Y_test = data
X_train = X_train / 255
Y_train = eznf.one_hot(Tensor(Y_train), 10)
X_train = Tensor([i.flatten() for i in X_train], requires_grad=False).T

In [ ]:
def zero_grad(m):
    for w in m.parameters():
        w.grad = None

def SGD(m: eznf.nn.Module, alpha):
    # 梯度下降
    for w in m.parameters():
        w.item = w.item - alpha*w.grad.item

In [ ]:
epoches = 100
m = test()
loss = eznf.nn.CrossEntropyLoss()
with tqdm(total=epoches) as t:
    for i in range(epoches):
        y = m(X_train)
        l = loss(y, Y_train.T) / 60000
        l.backward()
        SGD(m, 0.1)
        zero_grad(m)
        
        t.set_description('Epoch {}'.format(i), refresh=False)
        t.set_postfix(loss=l.item[0], refresh=False)
        t.update(1)

In [ ]:
l.grad_fn.a.grad_fn.x.grad_fn.a